In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.linear_model import LogisticRegression



In [3]:
res= pd.read_csv('/content/Dataset .csv')
pd.set_option('display.max_columns', None)


In [ ]:
res.columns

In [ ]:
res['distance'] = np.sqrt(res['Longitude']*res['Longitude'] + res['Latitude']*res['Latitude'])

In [ ]:
res['r_score'] = res['Aggregate rating']*res['Votes']
#'Restaurant Name','City','Currency','Cuisines'
#[res['Cuisines'] == 'Cafe']

In [ ]:
res['Cuisines'] = res['Cuisines'].fillna('').str.split(', ')

In [ ]:
ser = res[['Restaurant Name','City','Currency','Country Code','r_score','Cuisines']]

In [ ]:
ser = ser.explode('Cuisines', ignore_index=True)

In [ ]:
ohe = OneHotEncoder()
en_fe = ohe.fit_transform(ser[['Restaurant Name','City','Currency']])
#pd.DataFrame(en_fe.toarray() , columns=ohe.get_feature_names_out())
features =  pd.DataFrame(en_fe.toarray() , columns=ohe.get_feature_names_out())


In [ ]:
features.join(ser[['Country Code','r_score']],how='right')

In [ ]:
ser[['Country Code','r_score']].describe()

In [ ]:
from sklearn.preprocessing import LabelEncoder
lb = LabelEncoder()
ser['Cuisines'] = lb.fit_transform(ser['Cuisines'])

In [ ]:
from sklearn.model_selection import train_test_split

X = ser[['Country Code','r_score']]
y = ser['Cuisines']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [1]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import classification_report

clf = RandomForestClassifier()

clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)

print(classification_report(y_test, y_pred))

NameError: name 'X_train' is not defined

In [ ]:
import joblib
joblib.dump(clf , 'cuision_pre_v1.1_RndmFrst.pkl')

In [ ]:
lr = LogisticRegression(max_iter=200)
lr.fit(X_train, y_train)
predictions = lr.predict(X_test)

In [ ]:
print(classification_report(y_test, y_pred))

In [ ]:
res['Cuisines'].describe()

AttributeError: 'list' object has no attribute 'values_count'

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Embedding, Dense, GlobalAveragePooling1D
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np

# Assuming df['Locality'] and df['Cuisines'] are the columns in your dataset

# 1. Tokenizing Locality (X) which has multiple values in a list
localities = res['Locality'].apply(lambda x: ' '.join(x) if isinstance(x, list) else str(x)).tolist()

# 2. Tokenizing Cuisines (Y) which has multiple values (multi-label classification)
cuisines = res['Cuisines'].apply(lambda x: x if isinstance(x, list) else [x]).tolist()

# Create Tokenizer for Locality
tokenizer_locality = Tokenizer(num_words=1000, oov_token="<OOV>")
tokenizer_locality.fit_on_texts(localities)
sequences_locality = tokenizer_locality.texts_to_sequences(localities)

# Padding sequences for locality
padded_localities = pad_sequences(sequences_locality, padding='post')

# Create Tokenizer for Cuisines (multi-label binarization)
tokenizer_cuisines = Tokenizer(num_words=1000, oov_token="<OOV>")
tokenizer_cuisines.fit_on_texts(cuisines)
sequences_cuisines = tokenizer_cuisines.texts_to_sequences(cuisines)

# Binarize the cuisines (Y) for multi-label classification
cuisines_encoded = tf.keras.utils.to_categorical(sequences_cuisines)

# Ensure cuisines_encoded has a uniform shape
cuisines_encoded = np.array([np.sum(cuisine, axis=0) for cuisine in cuisines_encoded])

# Build a simple model
model = Sequential([
    Embedding(input_dim=1000, output_dim=16, input_length=padded_localities.shape[1]),
    GlobalAveragePooling1D(),
    Dense(24, activation='relu'),
    Dense(cuisines_encoded.shape[1], activation='sigmoid')  # Multi-label classification with sigmoid
])

# Compile the model with binary crossentropy for multi-label
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
model.fit(padded_localities, cuisines_encoded, epochs=10, batch_size=32)



In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Embedding, Dense, GlobalAveragePooling1D
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import MultiLabelBinarizer
import pandas as pd

# Assuming res is your dataframe
# 1. Flatten Locality values (if they're lists) to strings
res['Locality'] = res['Locality'].apply(lambda x: ' '.join(x) if isinstance(x, list) else str(x))

# 2. Split the Cuisines column into lists
res['Cuisines'] = res['Cuisines'].apply(lambda x: x.split(', ') if isinstance(x, str) else x)

# 3. Tokenize Locality (X)
locality_texts = res['Locality'].tolist()

tokenizer_locality = Tokenizer(num_words=1000, oov_token="<OOV>")
tokenizer_locality.fit_on_texts(locality_texts)
sequences_locality = tokenizer_locality.texts_to_sequences(locality_texts)

# Padding the Locality sequences
padded_localities = pad_sequences(sequences_locality, padding='post')

# 4. One-hot encode Cuisines (Y) using MultiLabelBinarizer
mlb = MultiLabelBinarizer()
cuisines_encoded = mlb.fit_transform(res['Cuisines'])

# 5. Build the model
model = Sequential([
    Embedding(input_dim=1000, output_dim=16, input_length=padded_localities.shape[1]),
    GlobalAveragePooling1D(),
    Dense(24, activation='relu'),
    Dense(cuisines_encoded.shape[1], activation='sigmoid')  # Multi-label output with sigmoid activation
])

# 6. Compile the model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# 7. Train the model
model.fit(padded_localities, cuisines_encoded, epochs=10, batch_size=32)


In [ ]:
# Evaluate the model on the training data (for testing, you'd normally use a separate test set)
loss, accuracy = model.evaluate(padded_localities, cuisines_encoded, verbose=2)
print(f"Loss: {loss}, Accuracy: {accuracy}")


In [ ]:
# New locality for prediction
new_locality = ["Moda"]  # Change this to any locality you want to predict

# Preprocess the new locality
new_locality_seq = tokenizer_locality.texts_to_sequences(new_locality)
new_locality_padded = pad_sequences(new_locality_seq, padding='post', maxlen=padded_localities.shape[1])

# Make prediction
predicted_cuisines = model.predict(new_locality_padded)

# Convert predictions back to readable cuisine labels
predicted_cuisine_labels = mlb.inverse_transform((predicted_cuisines > 0.5).astype(int))

print("Predicted Cuisines:", predicted_cuisine_labels)
